In [ ]:
library(tidyverse)
library(conflicted)
library(vegan)
library(grid)
library(gtable)
library(stringr)

In [ ]:
# Solve conflict
conflicts_prefer(dplyr::filter())

In [ ]:
# Load tables
bracken_raw = "/mnt/lustre/groups/maier/maina479/projects/test_metemgee/data/taxprofiler/taxpasta/bracken_B_standard_16gb.tsv" %>% 
    read_tsv() %>% 
    filter(str_detect(lineage, "Bacteria|Archaea"))

sample_data = "/mnt/lustre/groups/maier/maina479/projects/detaxizer/code/metadata_all.csv" %>% 
    read_delim(delim = ";")

In [ ]:
# Separate taxonomy into ranks
# Tax levels
tax_levels = c("superkingdom", "phylum", "class", "order",
               "family", "genus", "species")

# Create df with taxonomy data
taxonomy_raw =  bracken_raw %>% 
    select(lineage, rank_lineage, taxonomy_id)  

# The ranks are not uniform across taxa in this database
# Unify taxonomy using levels defined above
# Remove taxa without those levels from count table

# Create list of vectors with taxonomy and ranks
taxonomy_list = map2(taxonomy_raw$lineage, taxonomy_raw$rank_lineage, function(x, y){
    taxa = str_split_1(string = x,pattern = ";")
    ranks = str_split_1(string = y,pattern = ";")
    names(taxa) = ranks
    taxa[tax_levels]  %>% 
        as.data.frame %>% 
        rename("taxon" = ".")
})

# Create unified df with taxonomy and taxID
taxonomy_long = map2(taxonomy_list, taxonomy_raw$taxonomy_id, function(x, y){
    x %>%  
        rownames_to_column("rank") %>% 
        mutate(taxonomy_id = y)
}) %>% 
    list_rbind

In [ ]:
# Taxa without complete information
incomplete_taxonomy_IDs = taxonomy_long   %>% 
    filter(is.na(taxon) | is.na(rank) | rank %in% as.character(1:9)) %>% 
    pull(taxonomy_id) %>% 
    unique()


In [ ]:
# Filtered taxa df
taxonomy_filtered = taxonomy_long  %>% 
    filter(!(taxonomy_id %in% incomplete_taxonomy_IDs)) %>% 
    pivot_wider(id_cols = taxonomy_id, names_from = rank, values_from = taxon) %>% 
    mutate(taxonomy_id = as.character(taxonomy_id))

In [ ]:
# Wide table with relative abundances
bracken_relabund_wide = bracken_raw %>%  
    select(taxonomy_id, matches("_R1_filtered")) %>% 
    column_to_rownames("taxonomy_id")   %>% 
    vegan::decostand(method = "total", MARGIN = 2)  %>% 
    rownames_to_column("taxonomy_id") 

colnames(bracken_relabund_wide) = colnames(bracken_relabund_wide) %>% 
    str_remove_all("_R1_filtered.*")

In [ ]:
# abundance accounted by taxa with incomplete ranks
bracken_relabund_wide %>% 
    filter(taxonomy_id %in% incomplete_taxonomy_IDs) %>% 
    pivot_longer(cols = -taxonomy_id,names_to = "sample", values_to = "relabund") %>% 
    group_by(sample) %>% 
    summarise(sum_relabund = sum(relabund)*100) %>% 
    arrange(desc(sum_relabund)) %>% 
    head()

In [ ]:
# Long table with taxonomy
bracken_relabund_long = bracken_relabund_wide %>% 
    filter(!(taxonomy_id %in% incomplete_taxonomy_IDs)) %>% 
    pivot_longer(-taxonomy_id, names_to = "samples", values_to = "relative_abundance") %>% 
    left_join(taxonomy_filtered) %>% 
    left_join(sample_data)

bracken_relabund_long %>% 
    head()

In [ ]:
# Collapse abundances at a given taxonomic level
genus_relabund_long = bracken_relabund_long %>% 
filter(Group != "blank") %>% 
    group_by(genus, samples, Subject, Group) %>% 
    summarise(relative_abundance = sum(relative_abundance))

genus_relabund_long %>% 
    head()

In [ ]:
# Determine most abundant genus
msb_top_genus = genus_relabund_long %>% 
    filter(str_detect(Subject, "MSB")) %>% 
    group_by(genus) %>% 
    summarise(mean_relabund = mean(relative_abundance)) %>% 
    arrange(desc(mean_relabund)) %>% 
    slice(1:10)

msb_top_genus

In [ ]:
# Determine most abundant genus
com20_top_genus = genus_relabund_long %>% 
    filter(str_detect(Subject, "Com20")) %>% 
    group_by(genus) %>% 
    summarise(mean_relabund = mean(relative_abundance)) %>% 
    arrange(desc(mean_relabund)) %>% 
    slice(1:10)

com20_top_genus

In [ ]:
# Order of genus for plot
genus_levels = c(msb_top_genus$genus, "Other")

# Data frame for barplot
# Separate genus into top or not top for plot
# Calculate mean relative abundance per individual
msb_genus_df = genus_relabund_long %>% 
    filter(str_detect(Subject, "MSB")) %>% 
    group_by(genus, Subject, Group) %>% 
    summarise(subj_mean_relabund = mean(relative_abundance)) %>%
    mutate(top_genus = if_else(genus %in% msb_top_genus$genus, genus, "Other"), 
           top_genus = factor(top_genus, levels = genus_levels) ) %>% 
    arrange(Subject, desc(subj_mean_relabund))

msb_genus_df %>% 
    head()

In [ ]:
# Order of genus for plot
genus_levels = c(com20_top_genus$genus, "Other")

# Data frame for barplot
# Separate genus into top or not top for plot
# Calculate mean relative abundance per individual
com20_genus_df = genus_relabund_long %>% 
    filter(str_detect(Subject, "Com20")) %>% 
    group_by(genus, Subject, Group) %>% 
    summarise(subj_mean_relabund = mean(relative_abundance)) %>% 
    mutate(top_genus = if_else(genus %in% com20_top_genus$genus, genus, "Other"), 
           top_genus = factor(top_genus, levels = genus_levels) ) %>% 
    arrange(Subject, desc(subj_mean_relabund))

com20_genus_df %>% 
    head()

In [ ]:
# Color palette for MSB and Com20 
# Color palette with more differences according to: "https://stackoverflow.com/questions/9563711/r-color-palettes-for-many-data-classes"
c25 <- c(
  "#29bd16", "dodgerblue2", "#E31A1C", # red
  "#6A3D9A", # purple
  "#FF7F00", # orange
  "black", "gold1", # 6 and 7
  "skyblue2", "#FB9A99", # lt pink
  "#7eec7e", # palegreen2
  "#CAB2D6", # lt purple #11
  "#FDBF6F", # lt orange
  "gray70", "khaki2", #13 14
  "maroon", "orchid1", "deeppink1", "blue1", "#457aa7", #19 steelblue4
  "darkturquoise", "green1", "yellow4", "yellow3",
  "darkorange4", "brown" #24
)

# Select 14 colors for the drugs
drug_palette <- c25[c(1,2,3,4,7,5,8,9,15,10,16,17,18,24,20,25)]

c25 <- c(
  "#29bd16", "dodgerblue2", "#E31A1C", # red
  "#6A3D9A", # purple
  "#FF7F00", # orange
  "black", "gold1", # 6 and 7
  "skyblue2", "#FB9A99", # pink
  "#7eec7e", # palegreen2
  "#CAB2D6", # purple #11
  "#FDBF6F", # orange
  "gray70", "khaki2", #13 14
  "maroon", "orchid1", "deeppink1", "blue1", "#457aa7", #19 steelblue4
  "darkturquoise", "green1", "yellow4", "yellow3",
  "darkorange4", "brown" #24
)

# Select 14 colors for the drugs
drug_palette_com20 <- c25[c(2,15,3,17,23,14,4,10,19,13,16,17,18,24,20,25)]

In [ ]:
# MSB CBD Plots and Tables
msb_cbd_genus_df <- msb_genus_df %>%
  filter(str_detect(Group, "Cannabidiol")) %>%
  mutate(
    DrugLabel = str_trim(str_remove(Group, "\\d+(\\.\\d*)?")), # Remove concentration to extract drug name
    Concentration = as.numeric(str_extract(Group, "\\d+(\\.\\d*)?")), # Extract the concentration number from drug
    Treatment_order = factor(Group, levels = Group[order(Concentration)])) %>% # Order treatment by concentration
  ungroup()

# Calculate percentage of MSB CBD
msb_cbd_genus_percentages <- msb_cbd_genus_df %>%
  group_by(Treatment_order, Subject, top_genus) %>%
  summarise(
    genus_percent = 100 * sum(subj_mean_relabund, na.rm = TRUE),
    .groups = "drop")

# Save in csv
write_csv(msb_cbd_genus_percentages, "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/percentages_msb_cbd_genus.csv")

# MSB plot for CBD
msb_cbd_genus <- ggplot(msb_cbd_genus_df, aes(x = Subject, y = subj_mean_relabund, fill = top_genus)) +
  # Creates bar charts
  geom_col() +
  # 1 window row per treatment
  facet_wrap(~ Treatment_order) +
  # Colors for bar chart and legend
  scale_fill_manual(values = drug_palette, na.value = "grey50", name = "Genus") +
  labs(
    x = "Sample",
    y = "Relative abundance (%)") +
  # Design
  theme(
    legend.position = "bottom",
    text = element_text(size = 12),
    legend.key.size = unit(0.3, "cm"),
    axis.text.x = element_text(angle = 45, hjust = 1)) +   # Rotation of axis
  # Spaces adjusted
  scale_y_continuous(expand = c(0, 0), limits = c(0, NA)) +
  # Legend fixed
  guides(fill = guide_legend(nrow = 5))

# Save the plot
ggsave(filename = "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/msb_cbd_genus_abundance.png",
  plot = msb_cbd_genus, width = 12, height = 8, dpi = 300)


In [ ]:
# MSB Vortioxetine Plots and Tables
msb_vorti_genus_df <- msb_genus_df %>%
  filter(str_detect(Group, "Vortioxetine")) %>%
  mutate(
    DrugLabel = str_trim(str_remove(Group, "\\d+(\\.\\d*)?")), # Remove concentration to extract drug name
    Concentration = as.numeric(str_extract(Group, "\\d+(\\.\\d*)?")), # Extract the concentration number from drug
    Treatment_order = factor(Group, levels = Group[order(Concentration)])) %>% # Order treatment by concentration
  ungroup()

# Calculate percentage of MSB Vortioxetine
msb_vorti_genus_percentages <- msb_vorti_genus_df %>%
  group_by(Treatment_order, Subject, top_genus) %>%
  summarise(
    genus_percent = 100 * sum(subj_mean_relabund, na.rm = TRUE),
    .groups = "drop")

# Save in csv
write_csv(msb_vorti_genus_percentages, "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/percentages_vortioxetine_msb_genus.csv")

# MSB plot for Vortioxetine
msb_vorti_genus <- ggplot(msb_vorti_genus_df, aes(x = Subject, y = subj_mean_relabund, fill = top_genus)) +
  # Creates bar charts
  geom_col() +
  # 1 window row per treatment
  facet_wrap(~ Treatment_order) + 
  # Colors for bar chart and legend
  scale_fill_manual(values = drug_palette, na.value = "grey50", name = "Genus") +
  labs(
    x = "Sample",
    y = "Relative abundance (%)"
  ) +
  # Design
  theme(
    legend.position = "bottom",
    text = element_text(size = 12),
    legend.key.size = unit(0.3, "cm"),
    axis.text.x = element_text(angle = 45, hjust = 1)   # Rotation of axis
  ) +
  # Spaces adjusted
  scale_y_continuous(expand = c(0, 0), limits = c(0, NA)) +
  # Legend fixed
  guides(fill = guide_legend(nrow = 5))

# Save the plot
ggsave(filename = "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/msb_vortioxetine_genus_abundance.png",
  plot = msb_vorti_genus, width = 12, height = 8, dpi = 300)



In [ ]:

# Com20 CBD Plots and Tables
com20_cbd_df <- com20_genus_df %>% 
  filter(str_detect(Group, "Cannabidiol")) %>%
  mutate(
    DrugLabel = str_trim(str_remove(Group, "\\d+(\\.\\d*)?")), # Remove concentration to extract drug name
    Concentration = as.numeric(str_extract(Group, "\\d+(\\.\\d*)?")), # Extract the concentration number from drug
    Treatment_order = factor(Group, levels = Group[order(Concentration)]) # Order treatment
  ) %>%
  ungroup()

# Calculate percentage of Com20 CBD
com20_cbd_genus_percentages <- com20_cbd_df %>%
  group_by(Treatment_order, Subject, top_genus) %>%
  summarise(
    genus_abundance = sum(subj_mean_relabund, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  group_by(Treatment_order, Subject) %>%
  mutate(
    genus_percent = 100 * genus_abundance / sum(genus_abundance)
  ) %>%
  ungroup()

# Save in csv
write_csv(com20_cbd_genus_percentages, "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/cbd_com20_genus_percentages.csv")

# Com20 plot for CBD
com20_cbd <- ggplot(com20_cbd_df, aes(x = Subject, y = subj_mean_relabund, fill = top_genus)) +
  # Creates bar charts
  geom_col() +
  # 1 window row per for control
  facet_wrap(~ Treatment_order) +
  # Colors for bar chart and legend
  scale_fill_manual(values = drug_palette_com20, na.value = "grey50", name = "Genus") +
  labs(
    x = "Sample",
    y = "Relative abundance (%)"
  ) +
  # Design
  theme(
    legend.position = "bottom",
    text = element_text(size = 12),
    legend.key.size = unit(0.3, "cm")) + # Size for legend
  # Legend fixed
  guides(fill = guide_legend(override.aes = list(shape = 21, color = NA))) +
  # Spaces adjusted
  scale_y_continuous(expand = c(0, 0), limits = c(0, NA))

ggsave(filename = "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/com20_cbd_genus_abundance.png",
  plot = com20_cbd, width = 12, height = 8, dpi = 300)


In [ ]:
# Com20 Vortioxetine Plots and Tables
com20_vortiox_df <- com20_genus_df %>% 
  filter(str_detect(Group, "Vortioxetine")) %>%
  mutate(
    DrugLabel = str_trim(str_remove(Group, "\\d+(\\.\\d*)?")), # Remove concentration to extract drug name
    Concentration = as.numeric(str_extract(Group, "\\d+(\\.\\d*)?")), # Extract the concentration number from drug
    Treatment_order = factor(Group, levels = Group[order(Concentration)])) %>% # Order treatment
  ungroup()

# Calculate percentage of Com20 vortioxetine
com20_vortiox_genus_percentages <- com20_vortiox_df %>%
  group_by(Treatment_order, Subject, top_genus) %>%
  summarise(
    genus_abundance = sum(subj_mean_relabund, na.rm = TRUE),
    .groups = "drop") %>%
  group_by(Treatment_order, Subject) %>%
  mutate(
    genus_percent = 100 * genus_abundance / sum(genus_abundance)) %>%
  ungroup()

# Save in csv
write_csv(com20_vortiox_genus_percentages, "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/percentages_com20_vorti_genus.csv")

# Com20 plot for Vortioxetine
com20_vortiox <- ggplot(com20_vortiox_df, aes(x = Subject, y = subj_mean_relabund, fill = top_genus)) +
  # Creates bar charts
  geom_col() +
  # 1 window row per for control
  facet_wrap(~ Treatment_order) +
  # Colors for bar chart and legend
  scale_fill_manual(values = drug_palette_com20, na.value = "grey50", name = "Genus") +
  labs(
    x = "Sample",
    y = "Relative abundance (%)") +
  # Design
  theme(
    legend.position = "bottom",
    text = element_text(size = 12),
    legend.key.size = unit(0.3, "cm") ) + # Size for legend
  # Legend fixed
  guides(fill = guide_legend(override.aes = list(shape = 21, color = NA))) +
  # Spaces adjusted
  scale_y_continuous(expand = c(0, 0), limits = c(0, NA))

ggsave(filename = "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/vortioxetine_cbd_genus_abundance.png",
       plot = com20_vortiox, width = 12, height = 8, dpi = 300)

In [ ]:
# Com20 all abundance plots in one pdf
# New lables for df
com20_genus_all_df <- com20_genus_df %>%
  filter(Group != "blank" & Group != "DMSO control", !is.na(subj_mean_relabund)) %>%
  mutate(
    DrugLabel = str_trim(str_remove(Group, "\\d+(\\.\\d*)?")), # Remove concentration to extract drug name
    DrugLabel = factor(DrugLabel, levels = sort(unique(DrugLabel))), # Sort drugs alphabetically
    Concentration = as.numeric(str_extract(Group, "\\d+(\\.\\d*)?")), # Extract the concentration number from drug
    Conc_order = factor(Concentration, levels = sort(unique(Concentration))) # Ascending order for conc.
  )

# Takes the drug label and makes 3 rows of drugs per page ordered alphabetically
drug_names <- sort(unique(com20_genus_all_df$DrugLabel))
groups <- split(drug_names, ceiling(seq_along(drug_names) / 3))

# Opens a pdf
pdf("/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/com20_abundance_genus_all_drugs.pdf",
    width = 8.27, height = 11.69)

# Loop over all the groups
for (i in seq_along(groups)) {
  groups_drugs <- groups[[i]]

# Df is subseted to include the drugs of the set group
  groups_df <- com20_genus_all_df %>%
    filter(DrugLabel %in% groups_drugs) %>%
    droplevels()
  
# All plots will then be created and saved in one pdf
  all_com20_genus <- ggplot(groups_df, aes(x = Subject, y = subj_mean_relabund, fill = top_genus)) +
    # Creates bar charts
    geom_col() +
    # 3 windows row per drug label and concentration in ascending order
    facet_wrap(~ DrugLabel + Conc_order, ncol = 3, scales = "free_x") + 
    # Colors for bar chart and legend
    scale_fill_manual(values = drug_palette_com20, na.value = "grey50", name = "Genus") +
    labs(
      x = "Com20",    
      y = "Relative abundance (%)") +
    # Design of plot
    theme(
      panel.background = element_rect(fill = 'white', colour = 'black'),
      text = element_text(size = 14),
      legend.position = "right",
      strip.text = element_text(size = 10, face = "bold"),
      axis.text.x = element_blank(),     # No x-axis ticks labels
      axis.ticks.x = element_blank()) +    # No x-axis marks
    # Legend fixed
    guides(fill = guide_legend(override.aes = list(shape = 21, color = NA))) +
    # Spaces adjusted
    scale_y_continuous(expand = c(0, 0), limits = c(0, NA))
  
  print(all_com20_genus)
}

# The pdf is closed
dev.off()

In [ ]:
# Com20 all abundance plots in one pdf
# New lables for df
msb_genus_all_df <- msb_genus_df %>%
  filter(Group != "blank" & Group != "DMSO control", !is.na(subj_mean_relabund)) %>%
  mutate(
    DrugLabel = str_trim(str_remove(Group, "\\d+(\\.\\d*)?")), # Remove concentration to extract drug name
    DrugLabel = factor(DrugLabel, levels = sort(unique(DrugLabel))), # Sort drugs alphabetically
    Concentration = as.numeric(str_extract(Group, "\\d+(\\.\\d*)?")),# Extract the concentration number from drug
    Conc_order = factor(Concentration, levels = sort(unique(Concentration))) # Ascending order for conc.
  )
# Takes the drug label and makes 3 rows of drugs per page ordered alphabetically
drug_names <- sort(unique(msb_genus_all_df$DrugLabel))
groups <- split(drug_names, ceiling(seq_along(drug_names) / 3))

# Opens a pdf
pdf("/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/msb_abundance_genus_all_drugs.pdf",
    width = 8.27, height = 11.69)

# Loop over all the groups
for (i in seq_along(groups)) {
  chunk_drugs <- groups[[i]]

# Df is subseted to include the drugs of the set group
  groups_df <- msb_genus_all_df %>%
    filter(DrugLabel %in% chunk_drugs) %>%
    droplevels()

# All plots will then be created and saved in one pdf 
  all_msb_genus <- ggplot(groups_df, aes(x = Subject, y = subj_mean_relabund, fill = top_genus)) +
    # Creates bar charts
    geom_col() +
    # 3 windows row per drug label and concentration in ascending order
    facet_wrap(~ DrugLabel + Conc_order, scales = "free_x", ncol = 3) +
    # Colors for bar chart and legend
    scale_fill_manual(values = drug_palette, na.value = "grey50", name = "Genus") +
    labs(x = "Subject", y = "Relative abundance (%)") +
    # Design of plot
    theme(
      panel.background = element_rect(fill = 'white', colour = 'black'),
      text = element_text(size = 14),
      legend.position = "right",
      axis.text.x = element_blank(),     # No x-axis ticks labels
      axis.ticks.x = element_blank()) +     # No x-axis marks
    # Legend fixed
    guides(fill = guide_legend(override.aes = list(shape = 21, color = NA))) +
    # Spaces adjusted
    scale_y_continuous(expand = c(0, 0), limits = c(0, NA))
  
  # All groups are printed to the pdf
  print(all_msb_genus)
}

# The pdf is closed
dev.off()


In [ ]:
# MSB Control Plot
# Only controls and creates it's label
msb_genus_control_df <- msb_genus_df %>%
  filter(Group == "DMSO control", !is.na(subj_mean_relabund)) %>%
  mutate(DrugLabel = "DMSO control") %>%
  ungroup()

# Preparing percentages of abundance control
genus_percent_msb_control <- msb_genus_control_df %>%
  group_by(Subject, top_genus) %>%
  summarise(
    genus_percent = 100 * sum(subj_mean_relabund, na.rm = TRUE),
    .groups = "drop"
  )

# Save in csv file
write_csv(genus_percent_msb_control,
  "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/control_msb_genus_percentages.csv")

# Plot for the abundance genus of controls
abundance_control <- ggplot(msb_genus_control_df, aes(x = Subject, y = subj_mean_relabund, fill = top_genus)) +
  # Creates bar charts
  geom_col() +
  # 1 window row per for control
  facet_wrap(~ DrugLabel, ncol = 1, scales = "free_x") +  
  # Colors for bar chart and legend
  scale_fill_manual(values = drug_palette, na.value = "grey50", name = "Genus") +
  labs(
    x = "Subject",
    y = "Relative abundance (%)") +
  # Design of plot
  theme(
    panel.background = element_rect(fill = "white", colour = "black"),
    text = element_text(size = 14),
    legend.position = "right",
    strip.text = element_text(size = 10, face = "bold"),
    axis.text.x = element_text(angle = 45, hjust = 1)) + # No x-axis marks

    # Legend fixed
    guides(fill = guide_legend(override.aes = list(shape = 21, color = NA))) +
    # Spaces adjusted
    scale_y_continuous(expand = c(0, 0), limits = c(0, NA))

ggsave(filename = "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/control_msb_genus_plot.png",
  plot = abundance_control, width = 10, height = 6, dpi = 300)


In [ ]:
# Com20 Control Plot
# Only controls and creates it's label
com20_genus_control_df <- com20_genus_df %>%
  filter(Group == "DMSO control", !is.na(subj_mean_relabund)) %>%
  mutate(DrugLabel = "DMSO control") %>%
  ungroup()

# Preparing percentages of abundance control
genus_percent_com20_control <- com20_genus_control_df %>%
  group_by(Subject, top_genus) %>%
  summarise(
    genus_percent = 100 * sum(subj_mean_relabund, na.rm = TRUE),
    .groups = "drop"
  )

# Save in csv file
write_csv(genus_percent_com20_control,
  "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/control_com20_genus_percentages.csv")

# Plot for the abundance genus of controls
abundance_control_com20 <- ggplot(com20_genus_control_df, aes(x = Subject, y = subj_mean_relabund, fill = top_genus)) +
  # Creates bar charts
  geom_col() +
  # 1 window row per for control
  facet_wrap(~ DrugLabel, ncol = 1, scales = "free_x") +  # only one facet for control
  # Colors for bar chart and legend
  scale_fill_manual(values = drug_palette_com20, na.value = "grey50", name = "Genus") +
  labs(
    x = "Subject",
    y = "Relative abundance (%)") +
  # Design of plot
  theme(
    panel.background = element_rect(fill = 'white', colour = 'black'),
    text = element_text(size = 14),
    legend.position = "right",
    strip.text = element_text(size = 10, face = "bold")) +
  
  # Legend fixed
  guides(fill = guide_legend(override.aes = list(shape = 21, color = NA))) +
  # Spaces adjusted
  scale_y_continuous(expand = c(0, 0), limits = c(0, NA))

ggsave(filename = "/mnt/lustre/groups/maier/maina479/projects/abundance_plots/output/abundance_com20_control.png",
       plot = abundance_control_com20, width = 10, height = 6, dpi = 300)
